In [15]:
# Dependencies
import googlemaps
import pandas as pd

In [ ]:
gmaps = googlemaps.Client(key="SECRET")

In [47]:
bigfoot_df = pd.read_csv('data/filtered_years_clean.csv')

In [39]:
def geocode_with_fallback(row):
    try:
        # Try nearest town, state
        location = gmaps.geocode(f"{row['nearest town']}, {row['state']}")
        if location:
            return location[0]['geometry']['location']['lat'], location[0]['geometry']['location']['lng']

        # Fallback to county, state
        location = gmaps.geocode(f"{row['county']}, {row['state']}")
        if location:
            return location[0]['geometry']['location']['lat'], location[0]['geometry']['location']['lng']

        # If neither works, return None
        return None, None
    except Exception as e:
        print(f"Error geocoding {row['nearest town']}, {row['state']} or {row['county']}, {row['state']}: {e}")
        return None, None

In [14]:
bigfoot_stub = bigfoot_df.iloc[:10]

In [48]:
bigfoot_df[['Latitude', 'Longitude']] = bigfoot_df.apply(
    lambda row: pd.Series(geocode_with_fallback(row)), axis=1
)

In [51]:
bigfoot_df = bigfoot_df.drop(columns='Unnamed: 0')

In [52]:
bigfoot_df.to_csv('data/bigfoot_coords_df.csv')